In [1]:
# pip install roboflow

In [2]:
# Initializing model
from roboflow import Roboflow

rf = Roboflow(api_key="XG3i4cX7XdFeVFrfNqy5")
project = rf.workspace().project("dsc190-road-detection")

loading Roboflow workspace...
loading Roboflow project...


In [3]:
# Training specs:
# Roboflow Semantic Segmentation Model
# 640x400p pre-processing
# -15% and +15% brightness augmentations
model1 = project.version("10").model

# Training specs:
# Roboflow Semantic Segmentation Model
# 640x400p pre-processing
# -15% and +15% brightness augmentations
model2 = project.version("11").model

In [11]:
import cv2
import os
import shutil 
import time

def extract_frames_and_predict(video_path, output_folder, model):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        print("Error: Couldn't open the video file.")
        return

    # Remove all content if the folder already exists
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    
    # Create output folder
    os.makedirs(output_folder)

    # Initialize variables
    frame_count = 0

    # Read frames from the video, predict, and save the result
    while True:
        ret, frame = cap.read()

        if not ret:
            break  # Break the loop if there are no more frames

        frame_count += 1

        # Save each frame with a unique temporary file path
        temp_image_path = os.path.join(output_folder, f"temp_frame_{frame_count:04d}.jpg")
        cv2.imwrite(temp_image_path, frame)

        # Perform prediction on the saved frame
        prediction = model.predict(temp_image_path)

        # Save the prediction result as an image
        prediction_filename = os.path.join(output_folder, f"prediction_{frame_count:04d}.jpg")
        prediction.save(prediction_filename)
        
        # Release the resources associated with the video frame
        del frame

        # Add a small delay to ensure the file is released before attempting to remove it
        time.sleep(0.1) 

        # Remove the temporary file
        os.remove(temp_image_path)
        
        del temp_image_path
        del prediction
        del prediction_filename

    # Release the video capture object
    cap.release()

    print(f"Predictions saved to {output_folder}")

# Example usage
video_path = "P6010002_640_400.mp4"
output_folder = "predictions"

In [12]:
def combine_frames(folder_path, output_video, fps=30):

    # Get the list of JPG files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]

    # Sort the files based on their creation date
    image_files.sort(key=lambda x: os.path.getctime(os.path.join(folder_path, x)))

    # Get the first image to determine the dimensions of the video
    first_image_path = os.path.join(folder_path, image_files[0])
    first_image = cv2.imread(first_image_path)
    height, width, _ = first_image.shape

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use mp4v for .mp4 extension
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    # Iterate through the image files and write each frame to the video
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        frame = cv2.imread(image_path)
        video_writer.write(frame)

    # Release the VideoWriter object
    video_writer.release()

    print(f"Video created: {output_video}")
    
folder_path = "predictions"

In [13]:
extract_frames_and_predict(video_path, output_folder, model1)

Predictions saved to predictions


In [ ]:
output_video = "P6010002_Processed_1.mp4"
combine_frames(folder_path, output_video)

In [ ]:
extract_frames_and_predict(video_path, output_folder, model2)

In [ ]:
output_video = "P6010002_Processed_2.mp4"
combine_frames(folder_path, output_video)